#1. SQL: 

One Championship conducts events in a stadium, each event many people visit it and the stats are saved as these columns: id, event_name, people_count

Take leverage in adding sample data yourself.
Please write a query to display the records which have 3 or more consecutive events and the amount of people more than 100(inclusive).

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import DataFrame
from pyspark.sql import types
from pyspark.sql.functions import *
from pyspark.sql import Row
from pyspark.sql import SparkSession
l = [(1,'event1',100),(2,'event1',100),(3,'event1',100),(4,'event2',100),(5,'event2',100),(6,'event2',100),(7,'event1',100),(8,'event2',100)]
df=spark.createDataFrame(l, ['ID', 'event_name','people_count'])
df.createOrReplaceTempView("oneFcEvent")
ne=3      
spark.sql(f"""select distinct event_name,event_IDs from (select a.event_name,a.ID,lead(b.ID,{ne - 1}) over( partition by a.ID order by b.ID) as n_away, collect_list(b.ID) over ( partition by a.ID order by b.ID rows between current row and {ne - 1} following) as event_IDs from oneFcEvent a join oneFcEvent b on a.event_name = b.event_name and b.ID >= a.ID and b.people_count >= 100 where a.people_count >= 100) p where p.ID + {ne - 1} = n_away""").show()

+----------+---------+
event_name|event_IDs|
+----------+---------+
 event1|[1, 2, 3]|
 event2|[4, 5, 6]|
+----------+---------+

#2. Data Parsing: 

Write code to extract data from **data.csv**.

 

The data contains four columns. The first column is the person identifier. The second column is the datetime the person entered the floor. The third column is the floor the person accessed. The fourth column specifies the building the floor is in.

 

Each row is considered a floor access event. Your task it to output each floor access event in json format. Each event should comply with the schema located in **schema.json**.

In [4]:
from pyspark.sql.types import *
SCHEMA = StructType([
   StructField("person_id", StringType(), True),
   StructField("date_time", StringType(), True),
   StructField("floor_level", IntegerType(), True),
  StructField("building", StringType(), True)])

data_df= spark.read.schema(SCHEMA).option("header", "true").option("inferSchema","false").csv("/FileStore/tables/data.csv")
l=["person_id", "datetime", "floor_level", "building"]

from pyspark.sql.functions import coalesce, to_timestamp

def to_timestampconvert(col, formats=("MM/dd/yy HH:mm", "dd-MM-yyyy HH:mm:ss")):
  return coalesce(*[to_timestamp(col, f) for f in formats])

finaldf=data_df.withColumn("title",lit("Floor Access Event")).withColumn("type", lit("object")).withColumn("datetime", to_timestampconvert("date_time")). drop("date_time"). select("title", "type", struct(col("person_id"),col("datetime"),col("floor_level"),col("building")).alias("properties"))
display(finaldf)

title,type,properties
Floor Access Event,object,"List(1, 2015-10-01T08:02:00.000+0000, 6, B)"
Floor Access Event,object,"List(2, 2015-10-01T08:02:00.000+0000, 5, C)"
Floor Access Event,object,"List(3, 2015-10-01T08:03:00.000+0000, 1, C)"
Floor Access Event,object,"List(4, 2015-10-01T08:04:00.000+0000, 9, A)"
Floor Access Event,object,"List(5, 2015-10-01T08:04:00.000+0000, 6, B)"
Floor Access Event,object,"List(6, 2015-10-01T08:07:00.000+0000, 10, A)"
Floor Access Event,object,"List(7, 2015-10-01T08:07:00.000+0000, 9, C)"
Floor Access Event,object,"List(8, 2015-10-01T08:07:00.000+0000, 9, B)"
Floor Access Event,object,"List(9, 2015-10-01T08:07:00.000+0000, 10, C)"
Floor Access Event,object,"List(10, 2015-10-01T08:14:00.000+0000, 4, B)"


In [5]:
finaldf.coalesce(1).write.format('json').save("/FileStore/tables/output")